In [19]:
import os
import pandas as pd
from dotenv import load_dotenv
import dropbox
import requests
from utils import upload
import datetime
from zoneinfo import ZoneInfo

load_dotenv('../.env')

now_utc   = datetime.datetime.now(datetime.timezone.utc)
now_local = now_utc.astimezone(ZoneInfo("America/Montreal"))

def call_llm(system_prompt, user_prompt, print_reasoning=True):
    LLM_ENDPOINT   = "https://api.groq.com/openai/v1/chat/completions"
    LLM_API_KEY    = os.environ["GROQ_API_KEY"]
    LLM_MODEL      = "openai/gpt-oss-120b"

    REQ_TIMEOUT = 300  # seconds
    payload = {
        "model": LLM_MODEL,
        "messages": [

            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt,
            }
        ],
        # "temperature": 0.5,
        "max_tokens": 5000
    }

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {LLM_API_KEY}"
    }

    resp = requests.post(LLM_ENDPOINT, headers=headers, json=payload, timeout=REQ_TIMEOUT)
    resp.raise_for_status()
    resp_json = resp.json()

    # Adjust extraction to your provider’s schema
    try:
        text = resp_json["choices"][0]["message"]["content"].strip()
        reasoning = resp_json["choices"][0]["message"]["reasoning"].strip()
    except Exception as e:
        raise RuntimeError(f"Unexpected LLM response format: {resp_json}") from e

    if print_reasoning:
        print(reasoning)
        print("\n\n--------------------------\n\n")
    print(text)
    return text

# """You are an AI Emergency Department Operations assistant generating an Emergency Department (ED) Flow Report.
#             Describe yesterday's total inflow compared to what was expected, as well as a very brief comment about how busy or not the last week has been.
#             Also describe how many patients are waiting to be seen compared to what is expected.
#             Your target audience is a layperson with no understanding of statistics and modelling."""

# (f"Current date/time: {now_local} \n\n"
#             f"Recent daily inflow: \n{daily_inflow_markdown_table}\n\n"
#             f"Total patients waiting to see a doctor (TBS): \n {total_tbs_markdown_table}, ")

In [25]:
ed_metrics = {
    'INFLOW_STRETCHER': 'Hourly number of stretcher (non-ambulatory) patient arrivals to triage.',
    'Infl_Stretcher_cum': 'Cumulative stretcher inflow count since midnight.',
    'INFLOW_AMBULATORY': 'Hourly number of ambulatory (walk-in) patient arrivals to triage.',
    'Infl_Ambulatory_cum': 'Cumulative ambulatory inflow count since midnight.',
    'Inflow_Total': 'Total hourly patient arrivals (stretcher + ambulatory).',
    'Inflow_Cum_Total': 'Cumulative total inflow since midnight.',
    'INFLOW_AMBULANCES': 'Hourly number of arrivals brought by ambulance.',
    'Infl_Ambulances_cum': 'Cumulative ambulance arrivals since midnight.',
    'FLS': 'Number of patients presenting with flu-like symptoms.',
    'CUM_ADMREQ': 'Cumulative number of patients with an admission request.',
    'CUM_BA1': 'Cumulative number of admitted patients who have had a ward bed assigned and will therefore soon be transferred to a ward and leave the emergency department.',
    'WAITINGADM': 'Number of admitted patients currently waiting for an inpatient bed.',
    'TTStr': 'Total number of patients occupying a stretcher.',
    'TRG_HALLWAY1': 'Number of patients in the Prepod area awaiting a pod stretcher to be assigned. This is an overflow area for patients recently arriving.',
    'TRG_HALLWAY_TBS': 'Number of Prepod patietns waiting to be seen',
    'reoriented_cum': 'Cumulative number of reoriented (redirected) patients since midnight.',
    'reoriented_cum_MD': 'Do not use this metric.',
    'QTRACK1': 'Number of patients currently in QTrack (fast-track).',
    'RESUS': 'Number of patients currently in resuscitation rooms.',
    'Pod_T': 'Total number of patients in all pods (treatment areas).',
    'POD_GREEN': 'Number of patients in Green Pod.',
    'POD_GREEN_TBS': 'Number of Green Pod patients waiting to be seen.',
    'POD_YELLOW': 'Number of patients in Yellow Pod.',
    'POD_YELLOW_TBS': 'Number of Yellow Pod patients waiting to be seen.',
    'POD_ORANGE': 'Number of patients in Orange Pod.',
    'POD_ORANGE_TBS': 'Number of Orange Pod patients waiting to be seen.',
    'POD_CONS_MORE2H': 'Number of pod patients waiting more than 2 hours for a consult.',
    'POD_IMCONS_MORE4H': 'Number of pod patients waiting more than 4 hours for an internal medicine consult.',
    'POD_XRAY_MORE2H': 'Number of pod patients waiting more than 2 hours for X-ray.',
    'POD_CT_MORE2H': 'Number of pod patients waiting more than 2 hours for CT scan.',
    'POST_POD1': 'Number of patients in overflow stretcher areas that we call Family Rooms.',
    'VERTSTRET': 'Number of patients in vertical stretcher zone.',
    'RAZ_TBS': 'Number of RAZ (rapid assessment zone) patients waiting to be seen. In your report, use the acronym RAZ.',
    'RAZ_LAZYBOY': 'Number of patients in RAZ sitting/observation chairs. In your report, use the acronym RAZ.',
    'RAZ_WAITINGREZ': 'Number of RAZ patients waiting for results of investigations. In your report, use the acronym RAZ.',
    'AMBVERT1': 'Number of ambulatory patients.',
    'AMBVERTTBS': 'Number of ambulatory patients waiting to be seen.',
    'QTrack_TBS': 'Number of QTrack patients waiting to be seen.',
    'Garage_TBS': 'Number of patients in overflow/garage area waiting to be seen. We no longer use this metric.',
    'RAZ_CONS_MORE2H': 'Number of RAZ patients waiting more than 2 hours for a consult. In your report, use the acronym RAZ.',
    'RAZ_IMCONS_MORE4H': 'Number of RAZ patients waiting more than 4 hours for an internal medicine consult. In your report, use the acronym RAZ.',
    'RAZ_XRAY_MORE2H': 'Number of RAZ patients waiting more than 2 hours for X-ray. In your report, use the acronym RAZ.',
    'RAZ_CT_MORE2H1': 'Number of RAZ patients waiting more than 2 hours for CT scan. In your report, use the acronym RAZ.',
    'PSYCH1': 'Number of patients in the psychiatric zone.',
    'PSYCH_WAITINGADM': 'Number of psychiatric patients awaiting admission.',
    'total_tbs': 'Total number of patients currently to be seen across all zones.',
    'vert_tbs': 'Number of vertical (RAZ lazyboy/stretcher and ambulatory) patients currently to be seen.',
    'pod_tbs': 'Number of stretcher (pod) patients currently to be seen.',
    'overflow': 'Number of patients in overflow areas (garage, Prepod, and Postpod/Family Rooms).'
}

ed_metrics_string = "\n".join([f"**{k}**: {v}" for k, v in ed_metrics.items()])



In [21]:
current_df = pd.read_csv('https://www.dropbox.com/scl/fi/4zmm1bpwtrhrqtvce8aqq/current.csv?rlkey=1wnfrlal5jefwkx4fg7jxl20o&raw=1')
current_df.ds = pd.to_datetime(current_df.ds)

anomaly_df = pd.read_csv('https://www.dropbox.com/scl/fi/fjz0am427gw35sz7l994m/anomaly_detection_ranges.csv?rlkey=lib9w0jz2zei5n566jv76o7ol&raw=1')
anomaly_df.ds = pd.to_datetime(anomaly_df.ds)

target_column = 'Inflow_Total'
merged_df = pd.merge(current_df, anomaly_df, on='ds', how='left')
merged_df['anomaly'] = ((merged_df[target_column] < merged_df[target_column+'_yhat_lower']) | (merged_df[target_column] > merged_df[target_column+'_yhat_upper']))
merged_df[['ds', target_column, target_column+'_yhat',target_column+'_yhat_lower',target_column+'_yhat_upper', 'anomaly']]

# show only today's rows, in Montreal time zone
montreal_tz = 'America/Montreal'
today_montreal = pd.Timestamp.now(tz=montreal_tz).normalize()

# assign montreal timezone to 'ds' column
merged_df['ds_montreal'] = merged_df['ds'].dt.tz_localize(montreal_tz)
merged_df_today = merged_df[merged_df['ds_montreal'].dt.normalize() == today_montreal]
output_df = merged_df_today[['ds', target_column, target_column+'_yhat',target_column+'_yhat_lower',target_column+'_yhat_upper', 'anomaly']]


output_df.to_csv('hourly-inflow-today.csv', index=False)

# display as markdown table
print(output_df.to_markdown(index=False))


dropbox_app_key = os.environ.get("DROPBOX_APP_KEY")
dropbox_app_secret = os.environ.get("DROPBOX_APP_SECRET")
dropbox_refresh_token = os.environ.get("DROPBOX_REFRESH_TOKEN")

# exchange the authorization code for an access token:
token_url = "https://api.dropboxapi.com/oauth2/token"
params = {
    "grant_type": "refresh_token",
    "refresh_token": dropbox_refresh_token,
    "client_id": dropbox_app_key,
    "client_secret": dropbox_app_secret
}
r = requests.post(token_url, data=params)
# print(r.text)

dropbox_access_token = r.json()['access_token']

dbx = dropbox.Dropbox(dropbox_access_token)

upload(dbx, 'hourly-inflow-today.csv', '', '',
        'hourly-inflow-today.csv', overwrite=True)

| ds                  |   Inflow_Total |   Inflow_Total_yhat |   Inflow_Total_yhat_lower |   Inflow_Total_yhat_upper | anomaly   |
|:--------------------|---------------:|--------------------:|--------------------------:|--------------------------:|:----------|
| 2025-10-27 22:00:00 |              9 |            10.5632  |                  3.74772  |                  17.2818  | False     |
| 2025-10-27 21:00:00 |             22 |            11.6721  |                  5.22276  |                  17.7831  | True      |
| 2025-10-27 19:00:00 |             11 |            12.7366  |                  6.59703  |                  19.3009  | False     |
| 2025-10-27 18:00:00 |             14 |            13.524   |                  7.40068  |                  20.4039  | False     |
| 2025-10-27 17:00:00 |             15 |            14.714   |                  8.57455  |                  21.2956  | False     |
| 2025-10-27 16:00:00 |              9 |            16.0186  |                  9.6

FileMetadata(client_modified=datetime.datetime(2025, 10, 28, 2, 49, 51), content_hash='2a2b4f189c716a9b397e5e4bb5f1bcd3ca5f5af370f046d2e79fc47852cca372', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:oNSmVCFixyQAAAAAAABT0Q', is_downloadable=True, media_info=NOT_SET, name='hourly-inflow-today.csv', parent_shared_folder_id=NOT_SET, path_display='/hourly-inflow-today.csv', path_lower='/hourly-inflow-today.csv', preview_url=NOT_SET, property_groups=NOT_SET, rev='6422f11f4937b7a19c0a3', server_modified=datetime.datetime(2025, 10, 28, 2, 49, 52), sharing_info=NOT_SET, size=1943, symlink_info=NOT_SET)

In [22]:
allDataWithCalculatedColumns_df = pd.read_csv('https://www.dropbox.com/scl/fi/s83jig4zews1xz7vhezui/allDataWithCalculatedColumns.csv?rlkey=9mm4zwaugxyj2r4ooyd39y4nl&raw=1')
allDataWithCalculatedColumns_df.ds = pd.to_datetime(allDataWithCalculatedColumns_df.ds)
allDataWithCalculatedColumns_df.tail()

current = allDataWithCalculatedColumns_df.tail(1)
last_timestamp = allDataWithCalculatedColumns_df.iloc[-1].ds

df = allDataWithCalculatedColumns_df

anomaly_ranges = pd.read_csv('https://www.dropbox.com/scl/fi/fjz0am427gw35sz7l994m/anomaly_detection_ranges.csv?rlkey=lib9w0jz2zei5n566jv76o7ol&raw=1')
anomaly_ranges.ds = pd.to_datetime(anomaly_ranges.ds)
anomaly_ranges.tail(5)

def prepare_metric_table(target_metric):
    # current_target_metric = df.iloc[-1][target_metric]

    merged_current_with_anomaly_ranges = current.merge(anomaly_ranges, on='ds', how='left')

    metric_markdown_table = merged_current_with_anomaly_ranges[['ds',target_metric,target_metric+'_yhat',target_metric+'_yhat_lower',target_metric+'_yhat_upper']].to_markdown(index=False)
    # print(metric_markdown_table)

    return metric_markdown_table

tables = []
# for metric in ['total_tbs', 'pod_tbs', 'vert_tbs', 'overflow']:
for metric in ['INFLOW_STRETCHER', 'Infl_Stretcher_cum', 'INFLOW_AMBULATORY',
       'Infl_Ambulatory_cum', 'Inflow_Total', 'Inflow_Cum_Total',
       'INFLOW_AMBULANCES', 'Infl_Ambulances_cum', 'FLS', 'CUM_ADMREQ',
       'CUM_BA1', 'WAITINGADM', 'TTStr', 'TRG_HALLWAY1', 'TRG_HALLWAY_TBS',
       'reoriented_cum', 'reoriented_cum_MD', 'QTRACK1', 'RESUS', 'Pod_T',
       'POD_GREEN', 'POD_GREEN_TBS', 'POD_YELLOW', 'POD_YELLOW_TBS',
       'POD_ORANGE', 'POD_ORANGE_TBS', 'POD_CONS_MORE2H', 'POD_IMCONS_MORE4H',
       'POD_XRAY_MORE2H', 'POD_CT_MORE2H', 'POST_POD1', 'VERTSTRET', 'RAZ_TBS',
       'RAZ_LAZYBOY', 'RAZ_WAITINGREZ', 'AMBVERT1', 'AMBVERTTBS', 'QTrack_TBS',
       'Garage_TBS', 'RAZ_CONS_MORE2H', 'RAZ_IMCONS_MORE4H', 'RAZ_XRAY_MORE2H',
       'RAZ_CT_MORE2H1', 'PSYCH1', 'PSYCH_WAITINGADM', 'total_tbs', 'vert_tbs',
       'pod_tbs', 'overflow']:
    tables.append(prepare_metric_table(metric))

tables_string = '\n\n'.join(tables)
print(tables_string)

| ds                  |   INFLOW_STRETCHER |   INFLOW_STRETCHER_yhat |   INFLOW_STRETCHER_yhat_lower |   INFLOW_STRETCHER_yhat_upper |
|:--------------------|-------------------:|------------------------:|------------------------------:|------------------------------:|
| 2025-10-27 22:00:00 |                  6 |                 5.86337 |                       1.03181 |                       10.7744 |

| ds                  |   Infl_Stretcher_cum |   Infl_Stretcher_cum_yhat |   Infl_Stretcher_cum_yhat_lower |   Infl_Stretcher_cum_yhat_upper |
|:--------------------|---------------------:|--------------------------:|--------------------------------:|--------------------------------:|
| 2025-10-27 22:00:00 |                  159 |                   148.253 |                         114.524 |                         184.727 |

| ds                  |   INFLOW_AMBULATORY |   INFLOW_AMBULATORY_yhat |   INFLOW_AMBULATORY_yhat_lower |   INFLOW_AMBULATORY_yhat_upper |
|:--------------------|--

In [23]:
call_llm(
    system_prompt="You are an AI Emergency Department Operations assistant generating an Emergency Department (ED) Flow Report. "
                  "Summarize today's hourly inflow compared to what was expected. "
                  "Also describe how many patients are waiting to be seen compared to what is expected. Refer to this as TBS (Total patients waiting to be seen). "
                  "Your target audience is a layperson with no understanding of statistics and modelling."
                  "Summarize the data given in the table with key insights, no markdown tables in the output."
                  "Focus on anomalies and significant deviations from expected values, don't spend time on normal data points."
                  "Keep the response concise and to the point, minimal explanations, short bullets."
                  "Use simple language that a non-expert can understand."
                  "Don't refer to specific numbers unless they are particularly noteworthy."
                  "Don't refer to the model or prediction intervals, just focus on whether inflow was higher or lower than expected."
                  "Limit your response to 2 or 3 bullet points.",
    user_prompt=(f"Current date/time: {now_local} \n\n"
                f"Today's hourly inflow: \n{output_df.to_markdown(index=False)}\n\n"
                f"List of metrics and their descriptions: \n {ed_metrics_string}\n\n"
                f"Metrics with anomaly ranges: \n {tables_string}, "
                )
)

We need to produce concise bullet points (2-3) summarizing key insights: hourly inflow anomalies and TBS (total patients waiting to be seen). Identify notable deviations: anomalies flagged for Inflow_Total: at 21:00 (22 vs expected ~12) true anomaly, higher than expected. At 16:00 (9 vs expected 16) true anomaly, lower than expected. Also maybe other hours but those are flagged. Also maybe overall total waiting (total_tbs) is lower than expected (actual 17 vs expected 26, but within range? It is within lower bound 9.7 to 42.5, but lower than expected). So we can say TBS is lower than expected.

Also maybe vertical TBS is lower (actual 11 vs expected 20). Pod TBS is about as expected (6 vs 5.6). Overflow is lower (6 vs 10.7). So key insight: patients waiting lower than expected, especially overall.

We need to avoid numbers unless noteworthy. But we can mention "significantly higher at 9pm" and "significantly lower at 4pm". Also "overall waiting count is below what we usually see".

We 

'- The hour from\u202f9\u202fp.m. saw a sharp spike in arrivals – far more patients came in than we normally expect for that time of night.  \n- By\u202f4\u202fp.m. the flow slowed down noticeably – the number of new patients was well below the usual level for that hour.  \n- Overall, the total number of patients still waiting to be seen (TBS) is lower than usual, with fewer people in both the vertical (RAZ/ambulatory) and overflow areas than we typically have at this hour.'

In [26]:
final_report = call_llm(
    system_prompt="You are an AI Emergency Department Operations assistant generating an Emergency Department (ED) Flow Report. "
                  "Summarize today's hourly inflow compared to what was expected. "
                  "Also describe how many patients are waiting to be seen compared to what is expected. Refer to this as TBS (Total patients waiting to be seen). "
                  "Also highlight any other notable metrics that are outside anomaly ranges."
                  "Your target audience is a layperson with no understanding of statistics and modelling."
                  "Summarize the data given in the table with key insights, no markdown tables in the output."
                  "Focus on anomalies and significant deviations from expected values, don't spend time on normal data points."
                  "Keep the response concise and to the point, minimal explanations, short bullets."
                  "Use simple language that a non-expert can understand."
                  "Don't refer to specific numbers unless they are particularly noteworthy."
                  "Don't refer to the model or prediction intervals, just focus on the actual metric number and a qualitative description of how it fits in the expected range."
                #   "Limit your response to 5 highlights."
                  "Use markdown formatting for the output.",
    user_prompt=(f"Current date/time: {now_local} \n\n"
                f"Today's hourly inflow: \n{output_df.to_markdown(index=False)}\n\n"
                f"List of metrics and their descriptions: \n {ed_metrics_string}\n\n"
                f"Metrics with anomaly ranges: \n {tables_string}, "
                ), print_reasoning=False
)



**ED Flow Highlights – October 27 2025**

- **Hour‑by‑hour arrivals**
  - **9 PM:** Patient arrivals jumped well above the usual level (almost double the expected flow).  
  - **4 PM:** Arrivals were noticeably below what we normally see.

- **Total patients waiting to be seen (TBS)**
  - The overall queue of patients waiting for a clinician is **smaller than usual**, indicating the department is coping with the current load.

- **Areas with longer‑than‑usual waits**
  - **Pod consults:** More patients are waiting over 2 hours for a doctor’s consult than we typically expect.  
  - **Pod X‑ray:** The number waiting over 2 hours for an X‑ray is higher than normal.  
  - **Rapid‑assessment CT:** A surprisingly high count of patients are waiting over 2 hours for a CT scan in the rapid‑assessment zone.

- **Areas performing better than expected**
  - **Overall waiting list** (total TBS) and the **overflow zone** are both lower than usual, suggesting less crowding in the main ED flow.  
  - 

In [27]:
final_report = call_llm(
    system_prompt="You are an AI Emergency Department Operations assistant generating an Emergency Department (ED) Flow Report. "
                  "Summarize today's hourly inflow compared to what was expected. "
                  "Also describe how many patients are waiting to be seen compared to what is expected. Refer to this as TBS (Total patients waiting to be seen). "
                  "Also highlight any other notable metrics that are outside anomaly ranges."
                  "Your target audience is a layperson with no understanding of statistics and modelling."
                  "Summarize the data given in the table with key insights, no markdown tables in the output."
                  "Focus on anomalies and significant deviations from expected values, don't spend time on normal data points."
                  "Keep the response concise and to the point, minimal explanations, short bullets."
                  "Use simple language that a non-expert can understand."
                  "Don't refer to specific numbers unless they are particularly noteworthy."
                  "Don't refer to the model or prediction intervals, just focus on the actual metric number and a qualitative description of how it fits in the expected range."
                #   "Limit your response to 5 highlights."
                #   "Format your response as a json MessageCard to post to Teams via a webhook. There has to be a summary field."
                    "Use markdown formatting for the output. Do not include a title, I'll add it later.",
    user_prompt=(f"Current date/time: {now_local} \n\n"
                f"Today's hourly inflow: \n{output_df.to_markdown(index=False)}\n\n"
                f"List of metrics and their descriptions: \n {ed_metrics_string}\n\n"
                f"Metrics with anomaly ranges: \n {tables_string}, "
                ), print_reasoning=False
)



- **Hourly arrivals** – Two hours stood out: the 9 pm hour saw a sharp jump in patients (well above the usual range), while the 4 pm hour was unusually quiet (far fewer arrivals than expected).  

- **Cumulative admitted patients ready for a ward bed** – The total count of patients who have been assigned a ward bed is just below the normal lower limit, indicating fewer admissions ready for transfer than usual.  

- **Rapid‑assessment CT wait** – The number of patients in the rapid‑assessment zone waiting more than two hours for a CT scan is noticeably high, exceeding the expected maximum.  

- **Total patients waiting to be seen (TBS)** – The overall count of patients waiting across all zones is comfortably within the normal range but leans toward the lower side of that range, suggesting the department is not currently overwhelmed.  

- **Other areas** – Most other metrics (stretcher and ambulatory inflow, ambulance arrivals, resuscitation, pod occupancy, psychiatric load, etc.) are al

In [28]:
import requests
import json

# Replace with your Teams webhook URL
webhook_url = os.environ['HOURLY_AI_FLOW_REPORT_WEBHOOK']

# # Parse it into a Python dict
# payload = json.loads(final_report)

# # Send to Teams
# response = requests.post(webhook_url, json=payload)
# print(f"Status: {response.status_code}, Response: {response.text}")


In [ ]:
import requests
import json

# Replace with your Teams webhook URL
webhook_url = os.environ['HOURLY_AI_FLOW_REPORT_WEBHOOK']

adaptive_card = {
    "type": "message",
    "attachments": [
        {
            "contentType": "application/vnd.microsoft.card.adaptive",
            "contentUrl": None,
            "content": {
                "$schema": "https://adaptivecards.io/schemas/adaptive-card.json",
                "type": "AdaptiveCard",
                "version": "1.5",
                "body": [
                    {
                        "type": "TextBlock",
                        "text": "Hourly AI Flow Report",
                        "weight": "Bolder",
                        "size": "Large",
                        "wrap": True
                    },
                    {
                        "type": "TextBlock",
                        "text": final_report,
                        "wrap": True
                    }
                ]
            }
        }
    ]
}

requests.post(webhook_url, json=adaptive_card)

<Response [200]>